https://github.com/HKLeeeee/Multi_proj_6/blob/main/CNN%20model/model/multi-output/strawberry/sweep/Sweep%20Strawberry%20Model.ipynb

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.10

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [ ]:
!python --version

Python 3.7.14


In [ ]:
!pip install wandb

In [ ]:
import warnings 
warnings.filterwarnings(action='ignore')
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import inception_resnet_v2, nasnet
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback
import wandb

# Config

In [ ]:
train_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary_img/train'
valid_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary_img/valid'

CLASSES = ['normal', 'flooded']
NasNet = 'NasNet'
Inception = 'Inception'
target_size = {NasNet : (331,331),
               Inception : (299,299)}

TRAIN_LENGTH = len(os.listdir(train_dir))

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'pretrain_net': {
            'values': [Inception, NasNet]
        },
        'batch_size': {
            'values': [20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5]
        },
        'dense': {
            'values': [32, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range=30,
                              shear_range=0.2,
                              zoom_range=0.3,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
def train() :
    wandb.init()
    tf.keras.backend.clear_session()
    config_defaults = {
        'pretrain_net': Inception,
        'epochs' : 10,
        'batch_size': 50,
        'dropout' : 0.5,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam'
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_datagen.flow_from_directory(
        train_dir, # target directory
        classes = CLASSES,
        target_size=target_size[config.pretrain_net],
        batch_size=config.batch_size,
        class_mode='binary'
    )

    valid_generator = validation_datagen.flow_from_directory(
        valid_dir, # target directory
        classes = CLASSES,
        target_size=target_size[config.pretrain_net],
        batch_size=config.batch_size,
        class_mode='binary'
    )

    if config.pretrain_net == Inception :
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = target_size[config.pretrain_net]+(3,)
        )
    elif config.pretrain_net == NasNet :
        base_model= nasnet.NASNetLarge(
              weights='imagenet',
              include_top=False,
              input_shape = target_size[config.pretrain_net]+(3,)
             )
    base_model.trainable = False 

    inputs = layers.Input(shape=target_size[config.pretrain_net]+(3,))
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    x = layers.Dense(config.dense, activation=config.activation)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, 
                  outputs=outputs,
                  name='binary_flood')    
    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)


    model.compile(loss='cross',
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback()],
          steps_per_epoch=TRAIN_LENGTH//config.batch_size)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='flood24', entity='hkleeeee')

In [ ]:
wandb.agent(sweep_id, train)